# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770372771699                   -0.52    9.0         
  2   -2.771689391907       -2.88       -1.32    1.0    114ms
  3   -2.771714422232       -4.60       -2.46    1.0    118ms
  4   -2.771714631535       -6.68       -3.14    1.0    110ms
  5   -2.771714712136       -7.09       -3.78    2.0    106ms
  6   -2.771714715108       -8.53       -4.69    1.0    116ms
  7   -2.771714715249       -9.85       -5.43    2.0    109ms
  8   -2.771714715249      -12.74       -6.09    1.0    114ms
  9   -2.771714715250      -12.69       -7.10    2.0    121ms
 10   -2.771714715250   +  -14.10       -7.08    2.0    126ms
 11   -2.771714715250   +  -14.35       -6.92    1.0    107ms
 12   -2.771714715250      -15.35       -7.14    1.0    113ms
 13   -2.771714715250      -14.40       -7.93    1.0    101ms
 14   -2.771714715250   +  -14.45       -8.23    1.0    102ms


-0.00013457539725410746

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770379977123                   -0.53    8.0         
  2   -2.771771013293       -2.86       -1.31    1.0   88.0ms
  3   -2.771801667708       -4.51       -2.59    1.0    114ms
  4   -2.771802041337       -6.43       -3.70    1.0   91.4ms
  5   -2.771802072721       -7.50       -3.93    2.0    144ms
  6   -2.771802074385       -8.78       -4.82    1.0   94.6ms
  7   -2.771802074475      -10.05       -5.56    2.0    132ms
  8   -2.771802074476      -12.15       -6.03    2.0    113ms
  9   -2.771802074476      -13.62       -6.57    1.0    105ms
 10   -2.771802074476      -13.42       -7.14    2.0    124ms
 11   -2.771802074476   +  -13.70       -7.88    1.0    117ms
 12   -2.771802074476   +  -14.75       -9.33    2.0    124ms


0.01761222126908919

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457539725410746
Displaced dipole:  0.01761222126908919
Polarizability :   1.7746796666343294


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920877486e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766552024254e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852777078154e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694602811662e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088786968230e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.279894692725e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 8.938655191280e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 8.938655191280e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 6.768116045284e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.140406744600e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.342156663865e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.342156663865e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.342143130246661e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.